# Bob Environmental Visualization Workbook
A Jupyter Notebook that will let you import and visualize data that has been exported from the Bob Environmental App

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import pandas as pd
from datetime import datetime
import matplotlib.dates as mdates

# Import the AWU data loader - timestamps are automatically converted to datetime
import sys
import os
sys.path.append(os.getcwd())
from AWU_Importer import import_awu_data, transform_to_earth_coordinates, analyze_wave_motion

# Set up matplotlib for custom plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

kcolors = ['#007AFF','#C637A0', '#FF717C', '#1E0253']
plt.rcParams['axes.prop_cycle'] = cycler(color=kcolors)